<a href="https://colab.research.google.com/github/ychoi-kr/LLM-API/blob/main/openai/openai_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

## 파일 생성

https://platform.openai.com/storage 에서 업로드하거나 아래 코드를 실행

In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
training_file_dir = "/content/drive/MyDrive/llm-api-prog/data/"
training_file_name = "번역투순화_50례_20240320"

In [ ]:
client.files.create(
  file=open(training_file_dir + training_file_name + ".jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-JLGWuYxbeley6tBbXTxPDvkB', bytes=40298, created_at=1710919428, filename='번역투순화_50례_20240320.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
file_id = "file-JLGWuYxbeley6tBbXTxPDvkB"

## 작업 생성

https://platform.openai.com/finetune 에서 생성하거나 아래 코드를 실행

In [ ]:
client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-OAUwoa0ddecU7XerX2mfjW8R', created_at=1710919589, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-AmRROjbqJiSw6PeceyIUiQHY', result_files=[], status='validating_files', trained_tokens=None, training_file='file-JLGWuYxbeley6tBbXTxPDvkB', validation_file=None, user_provided_suffix=None)

## 작업 확인

https://platform.openai.com/finetune 에서 확인하거나 아래 코드를 실행

In [ ]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-OAUwoa0ddecU7XerX2mfjW8R', created_at=1710919589, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-AmRROjbqJiSw6PeceyIUiQHY', result_files=[], status='running', trained_tokens=None, training_file='file-JLGWuYxbeley6tBbXTxPDvkB', validation_file=None, user_provided_suffix=None)], object='list', has_more=False)

In [ ]:
jobname = "ftjob-OAUwoa0ddecU7XerX2mfjW8R"

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(jobname)

FineTuningJob(id='ftjob-OAUwoa0ddecU7XerX2mfjW8R', created_at=1710919589, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-AmRROjbqJiSw6PeceyIUiQHY', result_files=[], status='running', trained_tokens=None, training_file='file-JLGWuYxbeley6tBbXTxPDvkB', validation_file=None, user_provided_suffix=None)

In [ ]:
# 완료 후 모델 확인
client.fine_tuning.jobs.retrieve(jobname)

FineTuningJob(id='ftjob-OAUwoa0ddecU7XerX2mfjW8R', created_at=1710919589, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::94klBSwO', finished_at=1710920084, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-AmRROjbqJiSw6PeceyIUiQHY', result_files=['file-udvAke5IBpYzkURq0ZOAFBwc'], status='succeeded', trained_tokens=40881, training_file='file-JLGWuYxbeley6tBbXTxPDvkB', validation_file=None, user_provided_suffix=None)

In [ ]:
finetuned_model = "ft:gpt-3.5-turbo-0125:personal::94klBSwO"

## 테스트

In [ ]:
# 기본 모델
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "다음 문장을 자연스러운 한국어로 교정해 주세요."},
    {"role": "user", "content": "서비스 지향 아키텍처(service-oriented architecture, SOA)에 대한 아이디어는 새로운 것이 아니지만, 마이크로서비스 등장은 여러 옵션을 제공해주었다. 2000년대 초반에 SOA라는 용어가 도입되어 이미 이를 시도해보았다. 그러나 SOA의 경우, 다양한 요구사항과 제약 사항이 있었다. 통신 프로토콜부터 API 검색까지 사용량이 많은 애플리케이션까지 모든 것이 미리 결정되었거나 최소한 강력하게 권장되었다."}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='서비스 지향 아키텍처(SOA)에 대한 아이디어는 새로운 것이 아니었지만, 마이크로서비스의 등장으로 다양한 옵션이 제공되었습니다. 2000년대 초반에 SOA라는 용어가 나왔을 때 이미 이를 시도해 보았지만, 그 당시에는 다양한 요구사항과 제약조건이 있었습니다. 통신 프로토콜부터 API 검색까지, 사용빈도가 높은 애플리케이션의 모든 것이 미리 결정되었거나 적어도 강력히 권장되었던 것입니다.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
# 파인튜닝 모델
completion = client.chat.completions.create(
  model=finetuned_model,
  messages=[
    {"role": "system", "content": "다음 문장을 자연스러운 한국어로 교정해 주세요."},
    {"role": "user", "content": "서비스 지향 아키텍처(service-oriented architecture, SOA)에 대한 아이디어는 새로운 것이 아니지만, 마이크로서비스 등장은 여러 옵션을 제공해주었다. 2000년대 초반에 SOA라는 용어가 도입되어 이미 이를 시도해보았다. 그러나 SOA의 경우, 다양한 요구사항과 제약 사항이 있었다. 통신 프로토콜부터 API 검색까지 사용량이 많은 애플리케이션까지 모든 것이 미리 결정되었거나 최소한 강력하게 권장되었다."}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='서비스 지향 아키텍처 인 아이디어는 새롭지 않지만, 마이크로서비스가 등장하면서 여러 옵션을 제공하게 되었다. 2000년대 초반이라고 하면 SOA를 시도해본 시기이다. 하지만, SOA는 당시에 다양한 요구사항과 제약 사항이 있었다. 통신 프로토콜부터 API 검색까지 모든 것이 미리 결정되었거나 최소한 강력하게 권장되었다.', role='assistant', function_call=None, tool_calls=None)
